In [3]:
import os
from dotenv import load_dotenv
from smolagents import CodeAgent, LiteLLMModel

In [4]:
load_dotenv()

oci_user = os.getenv("OCI_USER")
oci_fingerprint = os.getenv("OCI_FINGERPRINT")
oci_tenancy = os.getenv("OCI_TENANCY")
oci_region = os.getenv("OCI_REGION")
oci_key = os.getenv("OCI_KEY")
oci_compartment_id = os.getenv("OCI_COMPARTMENT_ID")

# 必要な環境変数がすべて設定されているかチェック
required_vars = ['OCI_USER', 'OCI_FINGERPRINT', 'OCI_TENANCY', 'OCI_REGION', 'OCI_KEY', 'OCI_COMPARTMENT_ID']
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    print(f"警告: 以下の環境変数が設定されていません: {missing_vars}")

In [5]:
model = LiteLLMModel(
    #model_id="oci/meta.llama-3.3-70b-instruct",
    model_id="oci/meta.llama-4-maverick-17b-128e-instruct-fp8",
    #model_id="oci/xai.grok-4",
    # OCI固有のパラメータをkwargsとして渡す
    oci_region=os.getenv("OCI_REGION"),                    # 例: "us-ashburn-1"
    oci_user=os.getenv("OCI_USER"),                        # OCI User OCID
    oci_fingerprint=os.getenv("OCI_FINGERPRINT"),          # RSA key fingerprint
    oci_tenancy=os.getenv("OCI_TENANCY"),                  # Tenancy OCID
    oci_key=os.getenv("OCI_KEY"),                          # Private key content
    oci_compartment_id=os.getenv("OCI_COMPARTMENT_ID"),    # Compartment OCID
    temperature=0.0,
    max_tokens= 4000,
    #stream=True,
    drop_params=True  # 無効なパラメータを自動的に削除
)

In [6]:
agent = CodeAgent(
    tools=[], 
    model=model,
    # OCI用の設定: structured outputsを無効化（互換性向上のため）
    use_structured_outputs_internally=False,
    # 最大実行ステップ数を制限
    max_steps=10,
    stream_outputs=True
)



In [7]:
print(agent.prompt_templates["system_prompt"])

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of Thought, Code, and Observation sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the Code sequence you should write the code in simple Python. The code sequence must be opened with '{{code_block_opening_tag}}', and closed with '{{code_block_closing_tag}}'.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_